In [4]:
!pip install pyserial
!pip install torch
!pip install torchvision


  Using cached pyserial-3.5-py2.py3-none-any.whl.metadata (1.6 kB)
Using cached pyserial-3.5-py2.py3-none-any.whl (90 kB)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached torch-2.9.1-cp313-cp313-win_amd64.whl.metadata (30 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached markupsafe-3.0.3-cp313-cp313-win_amd64.whl.metadata (2.8 kB)
Using cached torch-2.9.1-cp313-cp313-win_amd64.whl (110.9 MB)
Using cached fsspec-2025.10.0-py3-none-any.whl (200 kB)
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached torchvision-0.24.1-cp313-cp313-win_amd64.whl.metadata (5.9 kB)
  Using cached pillow-12.0.0-cp313-cp313-win_amd64.whl.metadata (9.0 kB)
Using cached torchvision-0.24.1-cp313-cp313-win_amd64.whl (4.3 MB)
Using cached pillow-12.0.0-cp313-cp313-win_amd64.whl (7.0 MB)
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.8 MB 4.9 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/12.8 MB 4.8 MB/s eta 0:00:03
   --------- ------------------------------ 2.9/12.8 MB 4.8 MB/s eta 0:00:03
   ------------ --------------------------- 3.9/12.8 MB 4.8 MB/s eta 0:00:02
   --------------- ------------------------ 5.0/12.8 MB 4.9 MB/s eta 0:00:02
   ------------------ --------------------- 6.0/12.8 MB 4.8 MB/s eta 0:00:02
   ---------------------- ----------------- 7.1/12.8 MB 4.8 MB/s eta 0:00:02
   ------------------------- -------------- 8.1/12.8 MB 4.8 MB/s eta 0:00:01
   ---------------------------- --


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import serial
import csv
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset

In [3]:
import serial

import serial.tools.list_ports

def list_all_ports():
    ports = serial.tools.list_ports.comports()

    if not ports:
        print("No serial ports found.")
    else:
        print("Available serial ports:")
        for port in ports:
            print(f"Port: {port.device}, Description: {port.description}")

list_all_ports()

Available serial ports:
Port: COM6, Description: USB Serial Device (COM6)


In [4]:
serial_port = 'COM6';
baud_rate = 9600; #In arduino, Serial.begin(baud_rate)
ser = serial.Serial(serial_port, baud_rate)

In [5]:
ser.write('\n'.encode())

1

In [43]:
def processLine(line):
    line = line.split(',')
    line = line[0:4]
    if '' in line:
        return None
    if line == '':
        return None
    if len(line) != 4:
        return None
    line = [int(elem) for elem in line]
    line.append(line[0] - line[1])
    line.append(line[2] - line[3])
    return line

class SimpleNetWithHiddenLayer(nn.Module):
    def __init__(self):
        super(SimpleNetWithHiddenLayer, self).__init__()
        self.hidden = nn.Linear(6 * 15, 5)  # Input size 784, hidden layer size 128
        self.relu = nn.ReLU()                 # ReLU activation function
        self.output = nn.Linear(5, 5)      # Output layer for 10 classes (digits 0-9)

    def forward(self, x):
        x = x.view(-1, 6 * 15)  # Flatten the input image from (28, 28) to (784,)
        x = self.relu(self.hidden(x))  # Apply hidden layer and ReLU activation
        x = self.output(x)             # Apply output layer
        return x
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear = nn.Linear(6 * 15, 4)  # Input size 784, output size 10 (for digits 0-9)

    def forward(self, x):
        x = x.view(-1, 6 * 15)  # Flatten the image from (28, 28) to (784,1)

        return self.linear(x)


In [7]:
with open('hiddenlayermodel.pkl', 'rb') as f:
    model = pickle.load(f)
model.eval()

SimpleNetWithHiddenLayer(
  (hidden): Linear(in_features=90, out_features=5, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=5, out_features=5, bias=True)
)

In [25]:
def gesture(tensor):
    if (tensor == 0):
        return "Neural               UP"
    if (tensor == 1):
        return "Neural               LEFT"
    if (tensor == 2):
        return "Neural               DOWN"
    if (tensor == 3):
        return "Neural               RIGHT"
    if (tensor == 4):
        return ""

In [45]:
data = []
while (True):
    line = ser.readline()
    line = line.decode("utf-8") #ser.readline returns a binary, convert to string
    line = line.strip()
    if (len(line) > 20):
        print(line)
        continue
    processedLine = processLine(line)
    if processedLine == None:
        continue
    if (len(processedLine) < 4):
        continue
    if(len(data) >= 15):
        del data[0]
    
    data.append(processedLine)
    
    if(len(data) == 15):
        processed = np.array(data)
        processed = processed / 255.0
        processed = (processed - 0.5) / 0.5
        tensor = torch.tensor(processed, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        with torch.no_grad():
            output = model(tensor)
            ma, predicted_label = torch.max(output, 1)
            if (ma > 6):
                print(gesture(predicted_label))
                data = []

Neural               DOWN
Arduino Library Detected RIGHT gesture
Neural               RIGHT
Arduino Library Detected LEFT gesture
Arduino Library Detected RIGHT gesture
Neural               RIGHT
Arduino Library Detected LEFT gesture
Neural               LEFT
Arduino Library Detected RIGHT gesture
Neural               RIGHT
Arduino Library Detected DOWN gesture
Neural               LEFT
Arduino Library Detected UP gesture
Neural               RIGHT
Arduino Library Detected LEFT gesture
Neural               DOWN
Arduino Library Detected RIGHT gesture
Neural               RIGHT
Arduino Library Detected DOWN gesture
Neural               DOWN
Arduino Library Detected UP gesture
Neural               UP
Arduino Library Detected DOWN gesture
Neural               LEFT














Arduino Library Detected LEFT gesture
Neural               RIGHT
Arduino Library Detected RIGHT gesture
Neural               RIGHT
Arduino Library Detected LEFT gesture
Neural               LEFT
Arduino Library Dete

KeyboardInterrupt: 

In [15]:
while (True):
    line = ser.readline()
    line = line.decode("utf-8") #ser.readline returns a binary, convert to string
    line = line.strip()
    if (len(line) > 20):
        print(line)

Arduino Library Detected LEFT gesture
Arduino Library Detected UP gesture
Arduino Library Detected DOWN gesture
Arduino Library Detected RIGHT gesture
Arduino Library Detected DOWN gesture
Arduino Library Detected UP gesture
Arduino Library Detected LEFT gesture
Arduino Library Detected UP gesture
Arduino Library Detected DOWN gesture
Arduino Library Detected UP gesture
Arduino Library Detected DOWN gesture
Arduino Library Detected UP gesture
Arduino Library Detected DOWN gesture
Arduino Library Detected UP gesture
Arduino Library Detected LEFT gesture
Arduino Library Detected RIGHT gesture
Arduino Library Detected LEFT gesture
Arduino Library Detected RIGHT gesture
Arduino Library Detected DOWN gesture
Arduino Library Detected UP gesture
Arduino Library Detected LEFT gesture


KeyboardInterrupt: 